In [12]:
import random
import time
import turtle
import math
from copy import deepcopy

def heuristic(board, size):
    out = 0
    for i in range(size):
        for j in range(size):
            out += board[i][j]
    
    return out

def make_move(board, size, player, move):
    new_board = deepcopy(board)
    
    i, j = move
    new_board[i][j] = player
    for di in [-1, 0, 1]:
        for dj in [-1, 0, 1]:
            if di == 0 and dj == 0:
                continue
            x, y = i, j
            captured = []
            while 0 <= x + di < size and 0 <= y + dj < size and new_board[x + di][y + dj] == -player:
                captured.append((x + di, y + dj))
                x += di
                y += dj
            if 0 <= x + di < size and 0 <= y + dj < size and new_board[x + di][y + dj] == player:
                for (cx, cy) in captured:
                    new_board[cx][cy] = player

    return new_board

def get_valid_moves(board, size, player):
    moves = set()
    for i in range(size):
        for j in range(size):
            if board[i][j] == 0:
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        if di == 0 and dj == 0:
                            continue
                        x, y = i, j
                        captured = []
                        while 0 <= x + di < size and 0 <= y + dj < size and board[x + di][
                                y + dj] == -player:
                            captured.append((x + di, y + dj))
                            x += di
                            y += dj
                        if 0 <= x + di < size and 0 <= y + dj < size and board[x + di][
                                y + dj] == player and len(captured) > 0:
                            moves.add((i, j))
    return list(moves)

##############################################################

class OthelloUI:
    def __init__(self, board_size=6, square_size=60):
        self.board_size = board_size
        self.square_size = square_size
        self.screen = turtle.Screen()
        self.screen.setup(self.board_size * self.square_size + 50, self.board_size * self.square_size + 50)
        self.screen.bgcolor('white')
        self.screen.title('Othello')
        self.pen = turtle.Turtle()
        self.pen.hideturtle()
        self.pen.speed(0)
        turtle.tracer(0, 0)

    def draw_board(self, board):
        self.pen.penup()
        x, y = -self.board_size / 2 * self.square_size, self.board_size / 2 * self.square_size
        for i in range(self.board_size):
            self.pen.penup()
            for j in range(self.board_size):
                self.pen.goto(x + j * self.square_size, y - i * self.square_size)
                self.pen.pendown()
                self.pen.fillcolor('green')
                self.pen.begin_fill()
                self.pen.setheading(0)
                for _ in range(4):
                    self.pen.forward(self.square_size)
                    self.pen.right(90)
                self.pen.penup()
                self.pen.end_fill()
                self.pen.goto(x + j * self.square_size + self.square_size / 2,
                              y - i * self.square_size - self.square_size + 5)
                if board[i][j] == 1:
                    self.pen.fillcolor('white')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()
                elif board[i][j] == -1:
                    self.pen.fillcolor('black')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()

        turtle.update()

class Othello:
    def __init__(self, ui=False, minimax_depth=1, prune=True):
        self.size = 6
        self.ui = OthelloUI(self.size) if ui else None
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.board[int(self.size / 2) - 1][int(self.size / 2) - 1] = self.board[int(self.size / 2)][
            int(self.size / 2)] = 1
        self.board[int(self.size / 2) - 1][int(self.size / 2)] = self.board[int(self.size / 2)][
            int(self.size / 2) - 1] = -1
        self.current_turn = random.choice([1, -1])
        self.minimax_depth = minimax_depth
        self.prune = prune

    def get_winner(self):
        white_count = sum([row.count(1) for row in self.board])
        black_count = sum([row.count(-1) for row in self.board])
        if white_count > black_count:
            return 1
        elif white_count < black_count:
            return -1
        else:
            return 0

    def get_valid_moves(self, player):
        moves = set()
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            x, y = i, j
                            captured = []
                            while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == -player:
                                captured.append((x + di, y + dj))
                                x += di
                                y += dj
                            if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == player and len(captured) > 0:
                                moves.add((i, j))
        return list(moves)

    def make_move(self, player, move):
        i, j = move
        self.board[i][j] = player
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                x, y = i, j
                captured = []
                while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == -player:
                    captured.append((x + di, y + dj))
                    x += di
                    y += dj
                if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == player:
                    for (cx, cy) in captured:
                        self.board[cx][cy] = player

    def get_cpu_move(self):
        moves = self.get_valid_moves(-1)
        if len(moves) == 0:
            return None
        return random.choice(moves)

    ### ADDED ###
    def _value(self, board, player, depth, a = None, b = None):
        if depth == 0:
            return heuristic(board, self.size), None, 1
        
        if len(get_valid_moves(board, self.size, 1)) == 0 and len(get_valid_moves(board, self.size, -1)) == 0:
            return heuristic(board, self.size), None, 1
        
        best_move = None
        visiteds = 1
        v = 0

        if player == 1:
            v = -math.inf
        else:
            v = +math.inf

        for m in get_valid_moves(board, self.size, player):
            new_board = make_move(board, self.size, player, m)
            value, _m, _v = self._value(deepcopy(new_board), 1 if player == -1 else -1, depth-1, a, b)

            if player == 1:
                if value > v:
                    v = value
                    best_move = m

                if self.prune:
                    if value >= b:
                        v = value
                        best_move = m
                        break

                    a = max(a, value)

            else:
                if value < v:
                    v = value
                    best_move = m

                if self.prune:
                    if value <= a:
                        v = value
                        best_move = m
                        break
                    
                    b = min(b, value)

            visiteds += _v

        return v, best_move, visiteds
    
    ### EDITED ###
    def get_human_move(self):
        value, best_move, visisteds = self._value(deepcopy(self.board), 1, self.minimax_depth, -math.inf, math.inf)
        # if best_move == None: print("Cannot Move.")
        return best_move, visisteds

    def terminal_test(self):
        return len(self.get_valid_moves(1)) == 0 and len(self.get_valid_moves(-1)) == 0

    ### EDITED ###
    def play(self):
        winner = None
        visiteds = 0 # this line is added.
        
        while not self.terminal_test():
            if self.ui:
                self.ui.draw_board(self.board)
            if self.current_turn == 1:
                move, _v = self.get_human_move() # this line is edited (_v is added).
                # print("White:", move)
                if move:
                    self.make_move(self.current_turn, move)
                
                visiteds += _v # this line is added.
            else:
                move = self.get_cpu_move()
                # print("Black:", move)
                if move:
                    self.make_move(self.current_turn, move)
            self.current_turn = -self.current_turn
            if self.ui:
                self.ui.draw_board(self.board)
                time.sleep(1)
        winner = self.get_winner()
        return winner, visiteds # this line is edited (also return visiteds).

games = 100
depth = 1
purne = False

wins = 0
times = 0
total_visisteds = 0

for i in range(games):
    game = Othello(#ui=OthelloUI(),
                   minimax_depth=depth,
                   prune= purne)
    
    start_time = time.time()
    result, visisteds = game.play()
    end_time = time.time()

    result_str = "W" if result == 1 else "D" if result == 0 else "L"
    print(str(i) + ":", result_str + ", Time:", end_time-start_time, ", Visited Nodes:", visisteds)
    
    if result == 1: 
        wins += result
    
    times += end_time-start_time
    total_visisteds += visisteds

print("#################################")
print("Total Wins:", wins)
print("Average Time:", times/games)
print("َAverage Visited Nodes:", total_visisteds/games)

0: W, Time: 0.015105485916137695 , Visited Nodes: 101
1: L, Time: 0.01612234115600586 , Visited Nodes: 102
2: L, Time: 0.014400243759155273 , Visited Nodes: 79
3: W, Time: 0.01526641845703125 , Visited Nodes: 89
4: L, Time: 0.014417886734008789 , Visited Nodes: 84
5: W, Time: 0.015851974487304688 , Visited Nodes: 90
6: W, Time: 0.015511035919189453 , Visited Nodes: 94
7: W, Time: 0.01586318016052246 , Visited Nodes: 100
8: W, Time: 0.01513814926147461 , Visited Nodes: 98
9: W, Time: 0.014284610748291016 , Visited Nodes: 80
10: L, Time: 0.01679849624633789 , Visited Nodes: 99
11: W, Time: 0.022230863571166992 , Visited Nodes: 107
12: W, Time: 0.018949031829833984 , Visited Nodes: 104
13: L, Time: 0.016749858856201172 , Visited Nodes: 92
14: W, Time: 0.018741846084594727 , Visited Nodes: 122
15: W, Time: 0.01679825782775879 , Visited Nodes: 106
16: W, Time: 0.017373323440551758 , Visited Nodes: 96
17: L, Time: 0.0145721435546875 , Visited Nodes: 85
18: W, Time: 0.0197298526763916 , Visit